First: Number detection

In [ ]:
!git clone https://github.com/ahmed-nady/keras-ocr.git

Cloning into 'keras-ocr'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 856 (delta 2), reused 0 (delta 0), pack-reused 848
Receiving objects: 100% (856/856), 968.78 KiB | 26.91 MiB/s, done.
Resolving deltas: 100% (552/552), done.


*Install* Prereq

In [ ]:
!pip install  essential_generators 
!pip install validators
!pip install fonttools
!pip install editdistance
!pip install pyclipper
!pip install tqdm

Loading the fine-tuned model

In [ ]:
%cd /content/keras-ocr/
!gdown 'https://drive.google.com/uc?id=15fdLEOeUYZ8bnnj9tmsdlUvsCDm7sHgw' #detector_digit20_v11.h5


load data

In [ ]:
%cd /content/keras-ocr/

/content/keras-ocr


In [ ]:
#!gdown 'https://drive.google.com/uc?id=15ukczpqJa4j7Gb_GilzBcvhtXyTN8vBS' #Ice_hockey_Dataset
!gdown 'https://drive.google.com/uc?id=1MXnvzo4a_N0jtzlmNayxdZqAy-InfNUJ' #S_Basketball_Dataset
#!gdown 'https://drive.google.com/uc?id=1iH28eYogUDNX8eMKdlwQY7cHYmSszG2t'  #S_Basketball_Dataset_with_small_numer

#!gdown 'https://drive.google.com/uc?id=149hRNIKqN9o_cIZPnC-yb6RbyetjRrhQ' #combined_dataset_images_train_valid_test

In [ ]:
#!unzip  /content/keras-ocr/Ice_hockey_Dataset.zip
!unzip /content/keras-ocr/S_Basketball_Dataset.zip
#!unzip /content/keras-ocr/combined_dataset_images_train_valid_test.zip


In [ ]:
#!rm /content/keras-ocr/test/*
!rm /content/keras-ocr/output/*

In [ ]:
import matplotlib.pyplot as plt
import os
import keras_ocr
import cv2
import numpy as np

In [ ]:

from keras_ocr import detection
# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
detector = detection.Detector(weights_path='/content/keras-ocr/detector_digit20_v11.h5')

In [ ]:

import os.path

# Get a set of three example images
out_dir = '/content/keras-ocr/output/'
#======generalize test
img_dir = '/content/keras-ocr/S_Basketball_Dataset/'
img_path_list = []
img_name_lst = []
for entry in os.listdir(img_dir):
  if entry !='.ipynb_checkpoints':
    img_file_path = os.path.join(img_dir, entry)
    img_path_list.append(img_file_path)
    img_name_lst.append(entry)

count = 0
for img_path in img_path_list:
    #print("img_path",img_path)
    if not os.path.exists(img_path):
        count +=1
        continue 
    images = [
        keras_ocr.tools.read(url) for url in [img_path]]
    # # Make sure we have an image array to start with.
    if not isinstance(images, np.ndarray):
        images = [keras_ocr.tools.read(image) for image in images]
     
    # # # Make sure we have an image array to start with.
    # if not isinstance(images, np.ndarray):
    #     images = [cv2.merge([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)]) for image in images]
    
    # This turns images into (image, scale) tuples temporarily
    #we will use resize_aspect_ratio(img, square_size, instead of resize_image
    square_size =192
    images = [ keras_ocr.tools.resize_aspect_ratio(image, square_size,mag_ratio=1) for image in images ]
    ###========> resize_aspect_ratio_longest_side
    #images = [ keras_ocr.tools.resize_aspect_ratio_longest_side(image, square_size,mag_ratio=1) for image in images ]
    #images = [keras_ocr.tools.resize_image(image, max_scale=1, max_size=2048) for image in images]
    max_height, max_width = np.array([image.shape[:2] for image, scale in images]).max(axis=0)
    scales = [scale for _, scale in images]
    images = np.array(
        [keras_ocr.tools.pad(image, width=max_width, height=max_height) for image, _ in images])
    detection_kwargs ={'detection_threshold':0.1,'text_threshold':0.1,'link_threshold':0.1}
    if detection_kwargs is None:
        detection_kwargs = {}

    box_groups,score_map,link_map,rotation_angle =  detector.detect(images=images, **detection_kwargs)
    box_groups = [
        keras_ocr.tools.adjust_boxes(boxes=boxes, boxes_format='boxes', scale=1 /
                            scale) if scale != 1 else boxes
        for boxes, scale in zip(box_groups, scales)
    ]
    # # save results in txt files
    # count =0
    # for image, predictions in zip(images, box_groups):
    # result directory
    res_file = out_dir + "res_" + img_name_lst[count][:-4] + '.txt'
    #count +=1
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)

    with open(res_file, 'w') as f:
        for i, box in enumerate(box_groups[0]):
            poly = np.array(box).astype(np.int32).reshape((-1))
            strResult = ','.join([str(p) for p in poly]) + '\r\n' #','+str(rotation_angle)+ 
            f.write(strResult)

    image_cont = keras_ocr.tools.read(img_path_list[count])
    for box in box_groups[0]:
        cv2.polylines(img=image_cont,
                      pts=box[np.newaxis].astype('int32'),
                      color=(255, 100, 40),
                      thickness=2,
                      isClosed=True)
    
    #vis_img = keras_ocr.tools.drawBoxes(images[0],np.int32([box_groups[0]]),color=(255, 100, 40))
    #img_saved_name = out_dir +'frm_'+str(count)+'_'+'.jpg'
    img_saved_name = out_dir +img_name_lst[count][:-4]+'.jpg'
    score_map_saved_name = out_dir +img_name_lst[count][:-4]+'_score_txt.jpg'
    link_map_saved_name = out_dir +img_name_lst[count][:-4]+'_link.jpg'
    count +=1
    cv2.imwrite(img_saved_name,cv2.cvtColor(image_cont, cv2.COLOR_RGB2BGR))
    cv2.imwrite(score_map_saved_name,score_map)
    #cv2.imwrite(link_map_saved_name,link_map)


# **Number Recognition**

In [ ]:
%cd /content/
#!git clone https://github.com/clovaai/deep-text-recognition-benchmark
!git clone https://github.com/ahmed-nady/deep-text-recognition-benchmark.git
%cd deep-text-recognition-benchmark

/content
Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 459, done.
remote: Total 459 (delta 0), reused 0 (delta 0), pack-reused 459
Receiving objects: 100% (459/459), 3.04 MiB | 34.21 MiB/s, done.
Resolving deltas: 100% (280/280), done.
/content/deep-text-recognition-benchmark


Next, download large model files from Google Drive, using hack: https://stackoverflow.com/questions/20665881/direct-download-from-google-drive-using-google-drive-api/32742700#32742700

In [ ]:
models = {
    #'None-ResNet-None-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
    #'None-VGG-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1GGC2IRYEMQviZhqQpbtpeTgHO_IXWetG',
    #'None-VGG-None-CTC.pth': 'https://drive.google.com/open?id=1FS3aZevvLiGF1PFBm5SkwvVcgI6hJWL9',
    #'TPS-ResNet-BiLSTM-Attn-case-sensitive.pth': 'https://drive.google.com/open?id=1ajONZOgiG9pEYsQ-eBmgkVbMDuHgPCaY',
    'TPS-ResNet-BiLSTM-Attn.pth': 'https://drive.google.com/open?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9'#,
    #'TPS-ResNet-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
}

for k, v in models.items():
  doc_id = v[v.find('=')+1:]
  !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=$doc_id" > /tmp/intermezzo.html
  !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > $k

!ls -al *.pth

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3257    0  3257    0     0  13348      0 --:--:-- --:--:-- --:--:-- 13348
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   3264      0 --:--:-- --:--:-- --:--:--  3264
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  189M    0  189M    0     0  45.2M      0 --:--:--  0:00:04 --:--:-- 54.0M
-rw-r--r-- 1 root root 198678680 Dec 17 01:03 TPS-ResNet-BiLSTM-Attn.pth


In [ ]:
%cd /content/deep-text-recognition-benchmark
!mkdir demo_image_numbers

/content/deep-text-recognition-benchmark


# **Crop txt prediction bboxes**

In [ ]:
img_dir = '/content/keras-ocr/Ice_hockey_Dataset/'

In [ ]:
import shutil, os
import numpy as np
import cv2

pred_txt_dir = '/content/keras-ocr/output/'
saved_dir = '/content/deep-text-recognition-benchmark/demo_image_numbers/'
#img_dir = 'F:\\PhD\\Dataset\\ice_hockey_tracklets_5fps\\Ice_hockey_Dataset\\'

#img_dir = 'F:\\PhD\\Dataset\\combined_second_third_dataset\\dataset_Separation_small_nums_Gerke\\test\\'
# img_dir  ='F:\\PhD\\Dataset\\soccerDataset\\'
#img_dir = 'F:\\PhD\\Dataset\\gt_camera7_basketball_tracklets_5fps\\S_Basketball_Dataset\\'
count =0
for entry in os.listdir(pred_txt_dir):
	
    if entry.endswith('.txt'):
        txt_file_path = os.path.join(pred_txt_dir, entry)
        filename, file_ext = os.path.splitext(os.path.basename(txt_file_path))
        # temp = filename.split('_')[1:] 
        # image_id='_'.join(temp)
        image_path = img_dir+entry[4:-4]+'.png'
        #print(image_path)
        img = cv2.imread(image_path)

        if img is None:
            continue
        with open(txt_file_path) as f:
            lines = f.readlines() 
            count +=1
            for i,line in enumerate(lines):
                line = line.rstrip()
                rec_img_file = saved_dir + "rec_" + entry[4:-4] +'_'+str(i)+ '.png'
                points_coordinates = line.split(',')
                cnt = np.array([
                        [[int(points_coordinates[0]), int(points_coordinates[1])]],
                        [[int(points_coordinates[2]), int(points_coordinates[3])]],
                        [[int(points_coordinates[4]),int(points_coordinates[5])]],
                        [[int(points_coordinates[6]), int(points_coordinates[7])]]
                    ])

                x,y,w,h = cv2.boundingRect(cnt)
                img_h,img_w,_ = img.shape
                if x <0:
                  x=0
                if y < 0 :
                  y =0
                if x+w >img_w:
                  w = w-((x+w)-img_w)
                if y+h > img_h:
                  h = h -((y+h)-img_h)
                roi = img[y:y+h,x:x+w]
                #print("x,y,w,h",x,y,w,h)
                #print("img",img.shape)
                if roi is None:
                    continue
                #print(roi)
                cv2.imwrite(rec_img_file, roi)
        #print("count",count,image_path)

In [ ]:
%cd /content/deep-text-recognition-benchmark

/content/deep-text-recognition-benchmark


**TPS-ResNet-BiLSTM-Attn**

In [ ]:
output = !CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder demo_image_numbers/  --batch_size 1088 \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

# **Number detection and recognition evaluation**

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd
import os

In [ ]:
def get_predicted_text_v2(txt_dir,pred_txt_file_path):
	txt_pred =[]
	txt_pred_conf =[]

	df = pd.read_csv(pred_txt_file_path,sep="\t")
	#print(df)


	#get image names
	
	img_dict = {}

	for entry in os.listdir(txt_dir):
		if entry.endswith('.txt'):
			img_file_path = os.path.join(txt_dir, entry)
			filename, file_ext = os.path.splitext(os.path.basename(img_file_path))
			image_id='_'.join(filename.split('_')[1:])
			txt_save_file ='rec_'+image_id
	        #img_save_file ='rec_'+filename[3:]
			img_dict[txt_save_file] = ''
		else:
			txt_save_file ='rec_'+entry[0:-4]
			#img_save_file ='rec_'+filename[3:]
			img_dict[txt_save_file] = ''

	for ind, row in df.iterrows():
	  
	  # print("ind",ind)
	  # print("row",row)
	  number_flag = False
	  # if ind <3:
	  # 	continue
	  filename =  row[0]
	  label = row[1].strip()
	  conf = row[2]
	  # print("filename========",filename , (label),(conf))
	  # print("------",conf)
	  if label.isnumeric():
	  	#this means the detected text is number not string
	  	number_flag =True 

	  # make result file list
	  filenameWithoutExt, file_ext = os.path.splitext(os.path.basename(filename))
	  split_file_name = filenameWithoutExt.split('_')

	  img_dic_idx = filenameWithoutExt[:-2]
	   
	  #print(img_dic_idx)
	  #print("filenameWithoutExt.split('_')[2]",filenameWithoutExt.split('_'))
	  #print("split_file_name",split_file_name)
	  
	  if split_file_name[-1] =='0':
	  	 if number_flag :
		  	 txt_pred.append(label)
		  	 txt_pred_conf.append(conf)
		  	 img_dict[img_dic_idx] = label
	  	 else:
		  	 txt_pred.append('000')
		  	 txt_pred_conf.append(0)


	  	  
	  else:
	  	 #keep only one prediction per image
	  	 #print("txt_pred_conf",txt_pred_conf,conf,split_file_name)
	  	 if conf > txt_pred_conf[-1] and number_flag  : #conf >txt_pred_conf[-1] and number_flag
	  	 	txt_pred[-1] = label
	  	 	txt_pred_conf[-1] = conf
	  	 	img_dict[img_dic_idx] = label
	  	   

	#print("len of text prediction",len(txt_pred),txt_pred)
	#print("Img with text",img_dict)
	return img_dict

def get_ground_truth(img_dir):

	img_dict = {}

	for entry in os.listdir(img_dir):
		if entry.endswith('.png'):
			txt_save_file ='rec_'+entry[0:-4]
	        #img_save_file ='rec_'+filename[3:]
			imgClass = entry.split('_')[1]
			# # we use metrics for positive samples
			# if imgClass !='None':
			# 	img_dict[txt_save_file] =  imgClass
			img_dict[txt_save_file] =  imgClass if imgClass !='None' else ''

	return img_dict


In [ ]:
#img_dir = 'F:\\PhD\\Dataset\\gt_camera7_basketball_tracklets_5fps\\S_Basketball_Dataset_with_small_numer\\'
#txt_dir = 'F:\\PhD\\Dataset\\first_data_set\\gt\\'
rec_file = '/content/deep-text-recognition-benchmark/log_demo_result.txt'  
pred_img_dict = get_predicted_text_v2(img_dir,rec_file)
# get image class from their names
gt_img_dict = get_ground_truth(img_dir)
# #get image class from the content of text

# iterate over gt_img_dict
txt_pred =[]
txt_gt =[]
#missed_labels_count =0
for img_name,gt_label in gt_img_dict.items():
	# if gt_label =='' and pred_img_dict[img_name]=='':
	# 	missed_labels_count+=1
	#print("img_name========",img_name,gt_label,pred_img_dict[img_name])
	txt_gt.append(gt_label)
	txt_pred.append(pred_img_dict[img_name])

print("set(y_test) - set(y_pred)",set(txt_gt) - set(txt_pred))

#get Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy",accuracy_score(txt_gt, txt_pred))

actual_classes =list(set(txt_gt))
pred_classes = list(set(txt_pred))

class_labels = sorted(list(set(actual_classes+pred_classes)))
from sklearn.metrics import classification_report
print(classification_report(txt_gt, txt_pred,labels=np.unique(class_labels))) #labels=pred_classes

#get_eval_metric(txt_gt, txt_pred)
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
print(confusion_matrix(txt_gt, txt_pred))
# plot_confusion_matrix(None, txt_gt, txt_pred)  # doctest: +SKIP
# plt.show()
from sklearn.metrics import recall_score
print("====",recall_score(txt_gt, txt_pred, average='macro'))

print("micro",recall_score(txt_gt, txt_pred, average='micro'))
print("weighted",recall_score(txt_gt, txt_pred, average='weighted'))

set(y_test) - set(y_pred) {'18'}
Accuracy 0.8509544787077826
              precision    recall  f1-score   support

                   0.86      0.98      0.92       987
           0       0.00      0.00      0.00         0
           1       0.67      0.27      0.39        22
          10       0.88      0.39      0.54        18
         100       0.00      0.00      0.00         0
          11       0.67      0.16      0.26        25
         112       0.00      0.00      0.00         0
         115       0.00      0.00      0.00         0
          12       0.50      0.29      0.36         7
          13       0.94      0.63      0.75        46
         133       0.00      0.00      0.00         0
          14       0.80      0.67      0.73        18
          15       0.75      0.21      0.33        14
          17       0.92      0.76      0.83        29
          18       0.00      0.00      0.00         2
          19       0.88      0.93      0.90        15
           2       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!rm /content/deep-text-recognition-benchmark/demo_image_numbers/*
!rm /content/keras-ocr/output/*

rm: cannot remove '/content/deep-text-recognition-benchmark/demo_image_numbers/*': No such file or directory


Visualize results

In [ ]:
from IPython.core.display import display, HTML
from PIL import Image
import base64
import io
import pandas as pd

data = pd.DataFrame()
for ind, row in enumerate(output[output.index('image_path               \tpredicted_labels         \tconfidence score')+4:]):
  row = row.split('\t')
  if len(row) <2 or 'image_path'==row[0].strip():
    continue
  #print(row,type(row))
  filename = row[0].strip()
  label = row[1].strip()
  conf = row[2].strip()
  #print(filename)
  img = Image.open(filename)
  img_buffer = io.BytesIO()
  img.save(img_buffer, format="PNG")
  imgStr = base64.b64encode(img_buffer.getvalue()).decode("utf-8") 

  data.loc[ind, 'img'] = '<img src="data:image/png;base64,{0:s}">'.format(imgStr)
  data.loc[ind, 'id'] = filename
  data.loc[ind, 'label'] = label
  data.loc[ind, 'conf'] = conf

html_all = data.to_html(escape=False)
display(HTML(html_all))

**save output of detector**

In [ ]:
!zip -r /content/numbers_dataset_mask_v11_img_192.zip /content/keras-ocr/output

from google.colab import files
files.download('/content/numbers_dataset_mask_v11_img_192.zip')